In [1]:
from sklearn.model_selection import KFold

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import pathlib
import os

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers.legacy import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
import keras.backend as K
from sklearn.metrics import confusion_matrix
from PIL import Image
from IPython import display
from IPython.display import clear_output

%matplotlib inline

2024-02-04 13:11:38.167214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 13:11:38.167266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 13:11:38.167930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 13:11:38.173423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
BATCH_SIZE = 5
IMG_SIZE = (480, 480)
CLASS_NAMES = [ 'negativo', 'positivo']
classes_names = np.array([ 'neg', 'pos'])
train_path = pathlib.Path('./data/tmp/')

In [4]:
print(tf.io.gfile.listdir(train_path))
print(len(tf.io.gfile.listdir(train_path)))


['positivo', 'negativo']
2


In [5]:
list_ds = tf.data.Dataset.list_files(str(train_path/'*/*'))

for f in list_ds.take(5):
  print(f.numpy())

2024-02-04 13:11:39.993949: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 13:11:39.998752: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 13:11:39.998789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 13:11:40.001410: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 13:11:40.001460: I external/local_xla/xla/stream_executor

b'data/tmp/positivo/09 - pos_lam 897.jpg'
b'data/tmp/positivo/68-pos_lam 933.jpg'
b'data/tmp/positivo/1101.5_pos.jpg'
b'data/tmp/negativo/64-neg_lam 953.jpg'
b'data/tmp/positivo/1093.5_pos.jpg'


In [6]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def parse_image(filename):
  label = tf.strings.split(filename, os.sep)[-2]
  if label == 'positivo':
      label = 1.0
  else: 
      label = 0.0
  image = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, IMG_SIZE)
  return image, label
    
labeled_ds = list_ds.map(parse_image)

In [7]:
inputs = np.stack(list(labeled_ds.map(lambda x,y: x)))
targets = np.stack(list(labeled_ds.map(lambda x,y: y)))


In [8]:
num_folds = 3
acc_per_fold = []
loss_per_fold = []
loss_function = tf.keras.losses.BinaryCrossentropy()
optimizer = Adam(learning_rate=0.001)

In [ ]:
def create_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(480,480,3,)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  return model, model.get_weights()

def init_weights(same_model, weights):
  same_model.set_weights(weights)

model, weights = create_model()

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  init_weights(model, weights)


  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=BATCH_SIZE,
              epochs=20,
              verbose=2)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
# Merge inputs and targets
# inputs = tf.concat((input_train, input_test), axis=0)
# targets = tf.concat((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(480,480,3,)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))

  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=BATCH_SIZE,
              epochs=20,
              verbose=2)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

In [10]:
# prompt: create an input pipeline that implements kfold crossvalidation using a tensorflow dataset of images and labels

# Define the number of folds
num_folds = 5

# Create a list of indices for each fold
fold_indices = []
for i in range(num_folds):
  fold_indices.append(np.arange(i * len(labeled_ds) // num_folds, (i + 1) * len(labeled_ds) // num_folds))

# Create a list of tf.data.Datasets for each fold
fold_datasets = []
for i in range(num_folds):
  # Get the indices for the current fold
  test_indices = fold_indices[i]

  # Create a dataset for the test data
  test_dataset = labeled_ds.take(test_indices)

  # Create a dataset for the training data
  train_indices = np.setdiff1d(np.arange(len(labeled_ds)), test_indices)
  train_dataset = labeled_ds.skip(test_indices).take(train_indices)

  # Add the datasets to the list
  fold_datasets.append((train_dataset, test_dataset))

2024-02-04 13:16:04.932302: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at take_dataset_op.cc:227 : INVALID_ARGUMENT: count must be a scalar


InvalidArgumentError: {{function_node __wrapped__TakeDataset_device_/job:localhost/replica:0/task:0/device:CPU:0}} count must be a scalar [Op:TakeDataset] name: 

In [12]:
5*55

275

In [18]:
58/2

29.0